## Скачиваем необходимое

Сначала нужно средствами NLTK загрузить WordNet.



In [1]:
import nltk
from nltk.corpus import wordnet as wn
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to
[nltk_data]     /home/aleksandr/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

## Готовим данные к работе

Затем импортируем данные из подготовленного текстового файла. Файл содержит набор пар слов (только имён существительных), для которых известны экспертные оценки сходства.

Строим ассоциативный массив "пара слов -- оценка близости".

In [2]:
with open("wordsim_similarity_goldstandard.txt", encoding="utf-8") as rf:
  triples = [line.strip().split("\t") for line in rf.readlines()]
  score_map = {tuple(triple[:2]): float(triple[2]) for triple in triples}

In [23]:
score_map

{('tiger', 'cat'): 7.35,
 ('tiger', 'tiger'): 10.0,
 ('plane', 'car'): 5.77,
 ('train', 'car'): 6.31,
 ('television', 'radio'): 6.77,
 ('media', 'radio'): 7.42,
 ('bread', 'butter'): 6.19,
 ('cucumber', 'potato'): 5.92,
 ('doctor', 'nurse'): 7.0,
 ('professor', 'doctor'): 6.62,
 ('student', 'professor'): 6.81,
 ('smart', 'stupid'): 5.81,
 ('wood', 'forest'): 7.73,
 ('money', 'cash'): 9.15,
 ('king', 'queen'): 8.58,
 ('king', 'rook'): 5.92,
 ('bishop', 'rabbi'): 6.69,
 ('fuck', 'sex'): 9.44,
 ('football', 'soccer'): 9.03,
 ('football', 'basketball'): 6.81,
 ('football', 'tennis'): 6.63,
 ('Arafat', 'Jackson'): 2.5,
 ('physics', 'chemistry'): 7.35,
 ('vodka', 'gin'): 8.46,
 ('vodka', 'brandy'): 8.13,
 ('drink', 'eat'): 6.87,
 ('car', 'automobile'): 8.94,
 ('gem', 'jewel'): 8.96,
 ('journey', 'voyage'): 9.29,
 ('boy', 'lad'): 8.83,
 ('coast', 'shore'): 9.1,
 ('asylum', 'madhouse'): 8.87,
 ('magician', 'wizard'): 9.02,
 ('midday', 'noon'): 9.29,
 ('furnace', 'stove'): 8.79,
 ('food', 'frui

Отметим, что из исходного набора данных мы взяли только экспертные оценки сходства (similarity) и только для существительных. Исходный набор данных доступен по [ссылке](http://alfonseca.org/pubs/ws353simrel.tar.gz)

Посмотрим на примеры оценок. 

У слов может быть по несколько значений, которые различаются в WordNet. Здесь -- ради примера -- мы будем "жадно" выбирать первое попавшееся, но далее будем работать с ними иначе.




In [6]:
for w1, w2 in list(score_map)[:2]:
  
  print("\nWords: %s-%s\nGround truth score: %.2f" % (w1, w2, score_map[(w1, w2)]))
  
  ss1 = wn.synset(w1 + ".n.01")
  ss2 = wn.synset(w2 + ".n.01")

  print("\nPath: %.3f" % ss1.path_similarity(ss2), end=" ")
  print("\nwup: %.3f" % ss1.wup_similarity(ss2), end=" ")
  print("\nshortest_path: %.3f" % ss1.shortest_path_distance(ss2))


Words: tiger-cat
Ground truth score: 7.35

Path: 0.091 
wup: 0.545 
shortest_path: 10.000

Words: tiger-tiger
Ground truth score: 10.00

Path: 1.000 
wup: 0.750 
shortest_path: 0.000


Вычисляем для всех пар несколько оценок

In [7]:
from itertools import product

list_pairs = list(score_map)
wup_list, true_list, path_list = [], [], []

# для всех пар
for w1, w2 in list_pairs:

  try:
    all_w1 = wn.synsets(w1, pos="n")
    all_w2 = wn.synsets(w2, pos="n")

    # добавляем интересующие нас метрики и экспертные оценки
    wup = max([item1.wup_similarity(item2) \
                for item1, item2 in product(all_w1, all_w2)])
    wup_list.append(wup)

    path = max([item1.path_similarity(item2) \
                for item1, item2 in product(all_w1, all_w2)])
    path_list.append(path)
    
    true_list.append(score_map[(w1, w2)])

  except Exception as e:
    print(w1, w2, "error:", e)

drink eat error: max() arg is an empty sequence
stock live error: max() arg is an empty sequence


## Вычисляем ранговую корреляцию Спирмена

In [8]:
from scipy.stats import spearmanr

coef, p = spearmanr(wup_list, true_list)
print("wup  Spearman R: %.4f" % coef)

coef, p = spearmanr(path_list, true_list)
print("path Spearman R: %.4f" % coef)


wup  Spearman R: 0.6438
path Spearman R: 0.6176


## Факт

In [27]:
with open("Task_4_sample_1.csv", encoding="utf-8") as rf:
    triples = [line.strip().split("\t")[0].split(',') for line in rf.readlines()]
    score_map = {tuple(triple[:2]): float(triple[2]) for triple in triples[1:]}
  
score_map

{('television', 'radio'): 6.77,
 ('focus', 'life'): 4.06,
 ('vodka', 'brandy'): 8.13,
 ('experience', 'music'): 3.47,
 ('doctor', 'nurse'): 7.0,
 ('street', 'block'): 6.88,
 ('morality', 'importance'): 3.31,
 ('lad', 'wizard'): 0.92,
 ('Mexico', 'Brazil'): 7.44,
 ('precedent', 'example'): 5.85,
 ('month', 'hotel'): 1.81,
 ('prejudice', 'recognition'): 3.0,
 ('physics', 'chemistry'): 7.35,
 ('sugar', 'approach'): 0.88,
 ('bread', 'butter'): 6.19,
 ('man', 'governor'): 5.25,
 ('sign', 'recess'): 2.38,
 ('Wednesday', 'news'): 2.22,
 ('calculation', 'computation'): 8.44,
 ('consumer', 'confidence'): 4.13,
 ('life', 'term'): 4.5,
 ('stock', 'jaguar'): 0.92,
 ('street', 'avenue'): 8.88,
 ('stock', 'CD'): 1.31,
 ('start', 'match'): 4.47,
 ('volunteer', 'motto'): 2.56,
 ('stock', 'egg'): 1.81,
 ('precedent', 'group'): 1.77,
 ('psychology', 'psychiatry'): 8.08,
 ('profit', 'loss'): 7.63,
 ('tiger', 'mammal'): 6.85,
 ('tiger', 'tiger'): 10.0,
 ('computer', 'news'): 4.47,
 ('liquid', 'water'): 7.

In [31]:
list_pairs = list(score_map)
wup_list, true_list, lch_similarity_list, path_list = [], [], [], []

# для всех пар
for w1, w2 in list_pairs:

  try:
    all_w1 = wn.synsets(w1, pos="n")
    all_w2 = wn.synsets(w2, pos="n")

    # добавляем интересующие нас метрики и экспертные оценки
    wup = max([item1.wup_similarity(item2) \
                for item1, item2 in product(all_w1, all_w2)])
    wup_list.append(wup)

    path = max([item1.path_similarity(item2) \
                for item1, item2 in product(all_w1, all_w2)])
    path_list.append(path)
    
    lch_similarity = max([item1.lch_similarity(item2) \
                            for item1, item2 in product(all_w1, all_w2)])
    lch_similarity_list.append(lch_similarity)
    
    
    true_list.append(score_map[(w1, w2)])

  except Exception as e:
    print(w1, w2, "error:", e)

In [32]:
coef, p = spearmanr(wup_list, true_list)
print("wup  Spearman R: %.4f" % coef)

coef, p = spearmanr(lch_similarity_list, true_list)
print("lch_similarity  Spearman R: %.4f" % coef)

coef, p = spearmanr(path_list, true_list)
print("path Spearman R: %.4f" % coef)



wup  Spearman R: 0.6435
lch_similarity  Spearman R: 0.6107
path Spearman R: 0.6107


In [38]:
ss = wn.synset('bread.n.01')
(sorted([i.name() for i in ss.hyponyms()]), len(ss.hyponyms()))

(['anadama_bread.n.01',
  'bap.n.01',
  'barmbrack.n.01',
  'breadstick.n.01',
  'brown_bread.n.02',
  'bun.n.01',
  'caraway_seed_bread.n.01',
  'challah.n.01',
  'cinnamon_bread.n.01',
  'cracked-wheat_bread.n.01',
  'cracker.n.01',
  'crouton.n.01',
  'dark_bread.n.01',
  'english_muffin.n.01',
  'flatbread.n.01',
  'garlic_bread.n.01',
  'gluten_bread.n.01',
  'host.n.09',
  'loaf_of_bread.n.01',
  'matzo.n.01',
  'nan.n.04',
  'onion_bread.n.01',
  'quick_bread.n.01',
  'raisin_bread.n.01',
  'rye_bread.n.01',
  'salt-rising_bread.n.01',
  'simnel.n.01',
  'sour_bread.n.01',
  'toast.n.01',
  'wafer.n.03',
  'white_bread.n.01'],
 31)